In [ ]:
!pip -q install redis redis-cli

In [1]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)

## Search and Query in Redis with RediSearch - [Link to Video](YOUTUBE_LINK)

### Introduction

For Redis installations that require advanced querying, indexing, and full-text search capabilities, the **RediSearch** module extends Redis by providing a distributed querying engine. It delivers **real-time performance** with support for **high availability** and transforms Redis into a document-oriented search engine.

RediSearch enables developers to **collect, parse, and index data** for efficient retrieval using a powerful query language. It adds **document database features** on top of Redis core, making it possible to execute **numeric, tag, text, and geospatial queries**.

```python
# Example: Create a RediSearch index on Hashes with prefix "my:"
r.execute_command("FT.CREATE", "my:idx", "ON", "HASH", "PREFIX", "1", "my:",
                  "SCHEMA", "my_number", "NUMERIC", "SORTABLE",
                  "my_tag", "TAG",
                  "my_location", "GEO",
                  "my_text", "TEXT")
print(r.execute_command("FT._LIST"))  # verification of created index
````

> **Sidenote**
> [FT.CREATE | Docs - Redis](https://redis.io/docs/latest/commands/ft.create/?utm_source=chatgpt.com)
> **Command**: `FT.CREATE` → Creates an index in RediSearch.
> **Pattern**: `FT.CREATE index ON HASH|JSON PREFIX count prefix ... SCHEMA field TYPE [OPTIONS]`
> **Example**: `FT.CREATE myIdx ON HASH PREFIX 1 "doc:" SCHEMA title TEXT body TEXT`
> **Result**: Initializes a RediSearch index with the specified schema.

---

### Running RediSearch with Docker

To try RediSearch quickly, you can run it inside a Docker container:

```bash
docker run -p <PORT>:6379 redislabs/redisearch:latest
```

* `<PORT>` is the external port that maps Redis inside the container.
* Ensure the port does not conflict with other services.
* This exposes a Redis instance with RediSearch enabled.

---

### Defining a Schema

Before adding documents, define a schema that determines how fields are indexed:

```python
# Create an index with multiple fields and scoring
r.execute_command("FT.CREATE", "permits", "ON", "HASH", "PREFIX", "1", "tst:",
                  "SCORE_FIELD", "_score",
                  "SCHEMA",
                  "_score", "NUMERIC", "SORTABLE",
                  "permit_timestamp", "NUMERIC", "SORTABLE",
                  "address_street", "AS", "street", "TEXT", "NOSTEM",
                  "address_city", "AS", "city", "TAG", "SORTABLE",
                  "description", "TEXT",
                  "building_type", "TEXT", "WEIGHT", "20", "NOSTEM", "SORTABLE",
                  "work_type", "TAG", "SEPARATOR", ";",
                  "construction_value", "NUMERIC", "SORTABLE",
                  "location", "GEO")
print(r.execute_command("FT.INFO", "permits"))  # verification of schema
```

* The `_score` field is used to provide document scoring.
* If not set, the default score is `1`.
* Fields are mapped with aliases using `AS` for simpler query references.

---

### Indexing Basics

Adding documents into Redis Hashes automatically indexes them in RediSearch:

```python
# Add three permits with different attributes
r.execute_command("HSET", "tst:permit:1",
                  "description", "To reconstruct a single detached house with a front covered veranda.",
                  "construction_value", 42000,
                  "building_type", "single detached house",
                  "address_city", "Lisbon",
                  "work_type", "demolition,reconstruction",
                  "location", "-9.142685,38.736946")

r.execute_command("HSET", "tst:permit:2",
                  "description", "To construct a loft",
                  "construction_value", 53000,
                  "building_type", "apartment",
                  "address_city", "Porto",
                  "work_type", "construction",
                  "location", "-8.61099,41.14961")

r.execute_command("HSET", "tst:permit:3",
                  "description", "New house build",
                  "construction_value", 260000,
                  "building_type", "house",
                  "address_city", "Lagos",
                  "work_type", "construction;design",
                  "location", "-8.669586,37.129665")

print(r.execute_command("FT.SEARCH", "permits", "*"))  # verify documents are indexed
```

> **Sidenote**
> [HSET | Docs - Redis](https://redis.io/docs/latest/commands/hset/?utm_source=chatgpt.com)
> **Command**: `HSET` → Sets field-value pairs in a Redis Hash.
> **Pattern**: `HSET key field value [field value ...]`
> **Example**: `HSET user:1 name "Alice" age 30`
> **Result**: Stores multiple attributes under a single key.

---

### Querying Basics

#### Numeric Range Query

```python
# Find permits with exact construction value of 42000
print(r.execute_command("FT.SEARCH", "permits", "@construction_value:[42000 42000]"))
```

**Result**: Matches `tst:permit:1`.

---

#### Text Search

```python
# Search across all text fields for the word "veranda"
print(r.execute_command("FT.SEARCH", "permits", "veranda"))
```

**Result**: Matches `tst:permit:1`.

---

#### Tag Search

```python
# Search permits by city = "Lagos"
print(r.execute_command("FT.SEARCH", "permits", "@city:{Lagos}"))
```

**Result**: Matches `tst:permit:3`.
Note: Although the field is `address_city` in the Hash, the schema alias `AS city` allows queries with `@city`.

---

#### Geo Radius Search

```python
# Find permits near coordinates (-9.41563, 38.96275) within 50 km
print(r.execute_command("FT.SEARCH", "permits", "@location:[-9.41563 38.96275 50 km]"))
```

**Result**: Returns documents within 50 km radius, matching nearby cities.

> **Sidenote**
> [FT.SEARCH | Docs - Redis](https://redis.io/docs/latest/commands/ft.search/?utm_source=chatgpt.com)
> **Command**: `FT.SEARCH` → Executes queries on RediSearch indexes.
> **Pattern**: `FT.SEARCH index query [options]`
> **Example**: `FT.SEARCH permits "@price:[100 500]"`
> **Result**: Returns documents that match the query conditions.

---

## Sources

* [FT.CREATE | Docs - Redis](https://redis.io/docs/latest/commands/ft.create/?utm_source=chatgpt.com)
* [HSET | Docs - Redis](https://redis.io/docs/latest/commands/hset/?utm_source=chatgpt.com)
* [FT.SEARCH | Docs - Redis](https://redis.io/docs/latest/commands/ft.search/?utm_source=chatgpt.com)


### Original Transcript



Article - Search and Query
Search and Query Introduction
For the existing Redis install base who need to query their Redis datasets, RediSearch module provides a distributed querying, indexing, and full-text search engine that delivers real-time performance, with support for high-availability.

RediSearch extends Redis (Core) by a querying- , indexing, and full-text search engine, adding document database capabilities. RediSearch can collect, parse and store data for the purpose of information retrieval. It helps to optimize speed and performance in finding relevant data via a built-in query language.

Here is an example of how an index can be declared on Hashes that have a key with the prefix my::

FT.CREATE my:idx 

   ON HASH 

      PREFIX 1 "my:" 

   SCHEMA 

      "my_number" NUMERIC SORTABLE 

      "my_tag" TAG 

      "my_location" GEO 

      "my_text" TEXT

 

Running Search with Redis Hashes
You can either run the OSS version of RediSearch or create a Redis Enterprise database which has the module deployed. The simplest way to try RediSearch is to use the Docker container.

docker run -p <PORT>:6379 redislabs/redisearch:latest

Where <PORT> is the external port that makes the Redis Server inside the container available from the outside (make sure it does not conflict with existing ports in use). This command runs Redis with RediSearch and exposes RediSearch on the chosen port on the Docker host.

Defining a Schema
Before we can add entries (documents) to our search index, we need to define a schema:

FT.CREATE permits 

   ON HASH 

      PREFIX 1 "tst:" 

      SCORE_FIELD "_score" 

   SCHEMA 

      "_score" NUMERIC SORTABLE 

      "permit_timestamp" NUMERIC SORTABLE 

      "address_street" AS street TEXT NOSTEM 

      "address_city" AS city TAG SORTABLE 

      "description" TEXT 

      "building_type" TEXT WEIGHT 20 NOSTEM SORTABLE 

      "work_type" TAG SEPARATOR ";"

      "construction_value" NUMERIC SORTABLE 

      "location" GEO

Single-line version:

FT.CREATE permits ON HASH PREFIX 1 "tst:" SCORE_FIELD "_score" SCHEMA "_score" NUMERIC SORTABLE "permit_timestamp" NUMERIC SORTABLE "address_street" AS street TEXT NOSTEM "address_city" AS city TAG SORTABLE "description" TEXT "building_type" TEXT WEIGHT 20 NOSTEM SORTABLE "work_type" TAG SEPARATOR ";" "construction_value" NUMERIC SORTABLE "location" GEO

We use the field _score to provide the document score. If no value is set, then the score defaults to 1.

Indexing Basics
Let's add three documents to the index!

HSET tst:permit:1 "description" "To reconstruct a single detached house with a front covered veranda." "construction_value" 42000 "building_type" "single detached house" "address_city" "Lisbon" "work_type" "demolition,reconstruction" "location" "-9.142685,38.736946"

 

HSET tst:permit:2 "description" "To construct a loft" "construction_value" 53000 "building_type" "apartment" "address_city" "Porto" "work_type" "construction" "location" "-8.61099,41.14961"


HSET tst:permit:3 "description" "New house build" "construction_value" 260000 "building_type" "house" "address_city" "Lagos" "work_type" "construction;design" "location" "-8.669586,37.129665"

The hashes are indexed automatically behind the scenes.

Querying Basics
Numeric range queries:

FT.SEARCH permits "@construction_value:[42000,42000]"

Result: Every document that has a construction value of exactly 42000, so tst:permit:1.

Performing a text search on all text fields:

FT.SEARCH permits "veranda"

Result: Documents inside which the word 'veranda' occurs, so tst:permit:1.

Performing a tag search:

FT.SEARCH permits @city:{Lagos}

Result: Documents which have the address_city field set to "Lagos". Note that we use curly braces around the tag. Also note that even though the field is called address_city in the hash, we can query it as city. That's because in the schema definition we used the ... AS fieldname ... construct, which allowed us to index address_city as city.

Performing a geo radius search of the cities near to a location expressed as the pair represented by the longitude (first) plus the latitude (-9.41563,38.96275):

FT.SEARCH permits "@location:[-9.41563 38.96275 50 km]"

Result: Documents containing the cities near the chosen location (within a 50km radius).
